In [1]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [2]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'

In [3]:
path_save =""
winfo =[]
i=0
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]
            #rst = minp  - st
            #flt = ed - minp
            #maxp = base - np.max(pulse)
            #asy = ht / (ht + maxp)
            area = process_data.pusle_area(pulse, st, ed, base)
            area_fixlen = process_data.pulse_area_fix_len(pulse, st, 10, base)
            winfo.append({
                'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                #'Width': ed -st,  ## pusle width
                #'Minp': minp,     ## min point
                #'Rst' : rst,      ## rise time of pulse    
                #'Flt' : flt,      ## full time of pulse  
                #'Asym' : asy,     ## asymmetry of pulse
                'Area': area,     ## area of pulse cal by dynamic range
                'AreaFixlen': area_fixlen,    ## area of pulse cal by fix length of 10
                'Wave': pulse,
            })
            i +=1
            
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -13]  
        path_save = "outnpy/{}.h5".format(file_tag)

0it [00:00, ?it/s]/tmp/ipykernel_3346943/3053002512.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  asy = ht / (ht + maxp)
213270it [00:07, 30369.40it/s]
651372it [00:20, 31357.20it/s]


In [4]:
path_save

'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_11ns_400_ratio_run.h5'

In [5]:
df = pd.DataFrame(winfo)
process_data.write_to_hdf5(df, path_save)

/home/yjj/ProcessLocalData/process_data.py:56: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['Wave'], dtype='object')]

  df.to_hdf(filename, key='winfo', mode='w', complib='blosc:blosclz', complevel=9)


h5 Write Time: 5.583829402923584:.2f s 
h5 File Size: 291.3420248031616:.2f MB
Save to outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_11ns_400_ratio_run.h5


(5.583829402923584, 305494255)